<a href="https://colab.research.google.com/github/italomarcelogit/python.free/blob/master/COLAB/TENSORFLOW/Regressors/TensorFlow_Regressor_Tutorial_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/italomarcelogit/python.free/master/dataset-sales-random/dataset-sales-world.csv')

In [ ]:
dataset

,total_usd_million,month,year,country
0,5,1,2015,BR
1,5,2,2015,BR
2,7,3,2015,BR
3,9,4,2015,BR
4,8,5,2015,BR
...,...,...,...,...
283,7,8,2020,CAN
284,8,9,2020,CAN
285,7,10,2020,CAN
286,7,11,2020,CAN


# EDA

In [ ]:
dataset.isna().sum()

total_usd_million    0
month                0
year                 0
country              0
dtype: int64

In [ ]:
millionbyCountry = dataset[['total_usd_million', 'country']].groupby('country').sum()

In [ ]:
gc = px.pie(names=millionbyCountry.index, values=millionbyCountry.values,
            color_discrete_sequence=['lightblue', 'lightgreen', 'orange', 'pink'], 
            template="plotly_white")
gc.show()

In [ ]:
fig = px.bar(dataset, y='country', x='total_usd_million', 
             color="country", orientation="h", hover_name='year',
             color_discrete_map={
                "ARG": "lightblue",
                "BR": "green",
                "CAN": "pink",
                "PT": "darkred"},
             category_orders={"country": ["ARG", "BR", "CAN", "PT"]},
             title="2015 - 2020 (by month)", template='plotly_white'
            )

fig.show()

In [ ]:
from pandas.tseries.offsets import MonthEnd
datas = pd.to_datetime(dataset.year*10000+dataset.month*100+1,format='%Y%m%d')+ MonthEnd(1)

In [ ]:
df = dataset.copy()
df['date'] = datas

In [ ]:
ARG = dataset[dataset.country=='ARG']['total_usd_million']
BR = dataset[dataset.country=='BR']['total_usd_million']
CAN = dataset[dataset.country=='CAN']['total_usd_million']
PT = dataset[dataset.country=='PT']['total_usd_million']

df = pd.DataFrame(columns=['date', 'ARG', 'BR', 'CAN', 'PT'])
df.date = datas.unique()
df.ARG = ARG.values.tolist()
df.BR = BR.values
df.CAN = CAN.values
df.PT = PT.values

In [ ]:
df1 = df.melt(id_vars=['date']+list(df.keys()[5:]), var_name='C')
fig = px.line(df1, x='date', y='value', template="plotly_white",
              color='C',
              color_discrete_map={
                "ARG": "lightblue",
                "BR": "green",
                "CAN": "pink",
                "PT": "darkred"},
             category_orders={"C": ["ARG", "BR", "CAN", "PT"]},)
fig.show()

In [ ]:
def correlation(dfA, varT, minValue=0.5, 
                showGraphic=True, title='Correlation'):
    corr = dfA.corr()
    print(f'\nAnalysing features:\n'
          f'Target: {varT}\n'
          f'minValue de ref.: {minValue}\n'
          f'\nMain Features:')
    corrs = corr[varT]
    features = []
    for i in range(0, len(corrs)):
        if corrs[i] > minValue and corrs.index[i] != varT:
            print(corrs.index[i], f'{corrs[i]:.2f}')
            features.append(corrs.index[i])
    if showGraphic:     
        z_text = np.around(corr.values, decimals=2) 
        fig2 = ff.create_annotated_heatmap(corr.values, 
                                          annotation_text=z_text,
                                           x=corr.index.values.tolist(),
                                y=corr.columns.values.tolist(),
                                colorscale='Viridis',
                                hoverongaps = False)
        
        for i in range(len(fig2.layout.annotations)):
            fig2.layout.annotations[i].font.size = 8
            
        fig2.show()

        
        
    
    return features

In [ ]:
varTarget = 'total_usd_million'
varsFeatures = correlation(dataset, varT=varTarget, minValue=-0.3)


Analysing features:
Target: total_usd_million
minValue de ref.: -0.3

Main Features:
month -0.00
year 0.14
